In [3]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip uninstall transformers -y
!pip install -U "transformers==4.56.0" 

Found existing installation: transformers 4.56.0
Uninstalling transformers-4.56.0:
  Successfully uninstalled transformers-4.56.0
  Using cached transformers-4.56.0-py3-none-any.whl.metadata (40 kB)
Using cached transformers-4.56.0-py3-none-any.whl (11.6 MB)


In [5]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from tqdm import tqdm
import os, zipfile, torch

2025-11-17 13:00:14.204855: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763384414.406498      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763384414.461953      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [6]:
# === Константы ===
DATASET_NAME = "Lipsrow/ruwiki_cleaned"
MODEL_EMB = "sergeyzh/LaBSE-ru-turbo"
# === 1. Загрузка и подготовка данных ===
print("Загружаем датасет...")
ds = load_dataset(DATASET_NAME, split="train")

Загружаем датасет...


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

ruwiki.json:   0%|          | 0.00/7.70G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [8]:
!pip install llama-cpp-python

from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="IlyaGusev/saiga_yandexgpt_8b_gguf",
	filename="saiga_yandexgpt_8b.BF16.gguf",
    n_gpu_layers=-1, 
    device="cuda"
)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 28.5 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp311-cp311-linux_x86_64.whl size=4503256 sha256=4f8fd1b1d18c469aa8c3cb09afdf02aeb1eab6e19f1e2ca11c18b45eb114ebff
  Stored in directory: /root/.cache/pip/wheels/d8/5b/e5/a7d4b5765da347d314e8155197440c9995a962f8e4a5f52b23
Successfully built llama-cpp-python


./saiga_yandexgpt_8b.BF16.gguf:   0%|          | 0.00/16.1G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 33 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--IlyaGusev--saiga_yandexgpt_8b_gguf/snapshots/e902caf817aed403060016db9e35e21f59d1ec49/./saiga_yandexgpt_8b.BF16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Saiga_Yandexgpt_8B_Sft_M4_D19_Hf
llama_model_loader: - kv   3:                       general.organization str              = Models
llama_model_loader: - kv   4:                         general.size_label str              = 8.0B
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   

In [ ]:
llm.to(device)

In [9]:
texts = []
MAX_WORDS = 200
for text in tqdm(ds['text'], desc="Извлечение первых абзацев"):
    first_paragraph = text.strip().split("\n")[0]
    words = first_paragraph.split()
    if len(words) > MAX_WORDS:
        first_paragraph = " ".join(words[:MAX_WORDS])
    texts.append(first_paragraph)

Извлечение первых абзацев: 100%|██████████| 500000/500000 [00:25<00:00, 19760.42it/s]


In [13]:
import os, json
from tqdm import tqdm

out_path = "texts.jsonl"

with open(out_path, "w", encoding="utf-8") as f:
    for t in tqdm(texts, desc="write-texts"):
        para = t  # если уже первый абзац – можно убрать
        if not para:
            continue
        f.write(json.dumps({"text": para}, ensure_ascii=False) + "\n")

print("saved:", out_path)

write-texts: 100%|██████████| 500000/500000 [00:04<00:00, 106080.76it/s]

saved: texts.jsonl


In [ ]:
#restart

In [14]:
texts = []
with open("/kaggle/working/texts.jsonl", "r", encoding="utf-8") as f:
    for line in tqdm(f, desc="read-texts"):
        obj = json.loads(line)
        texts.append(obj.get("text", ""))
print("loaded:", len(texts))

read-texts: 500000it [00:03, 164122.26it/s]

loaded: 500000


In [19]:
 !nvidia-smi

Mon Nov 17 13:29:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P0             37W /  250W |    2571MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
!pip install -U --force-reinstall --no-cache-dir llama-cpp-python-cu121

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement llama-cpp-python-cu121 (from versions: none)
ERROR: No matching distribution found for llama-cpp-python-cu121


In [16]:
device = 'cuda'

In [17]:
# === 2. Создание эмбеддингов ===
print("Создаём эмбеддинги...")
embedder = SentenceTransformer(MODEL_EMB).to(device)
embeddings = []

for i in tqdm(range(0, len(texts), 256), desc="Векторизация"):
    batch = texts[i:i+256]
    vecs = embedder.encode(batch, convert_to_numpy=True, show_progress_bar=False, device = device,normalize_embeddings=True)
    embeddings.append(vecs)

embeddings = np.vstack(embeddings).astype("float32")

Создаём эмбеддинги...


Векторизация:  49%|████▉     | 955/1954 [17:19<18:07,  1.09s/it]


KeyboardInterrupt: 

In [24]:
embeddings = np.vstack(embeddings).astype("float32")

In [25]:
print("Создаём FAISS индекс...")
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)
#faiss.write_index(index, DB_PATH)

Создаём FAISS индекс...


In [26]:
SYSTEM_PROMPT = """Ты — эрудированный ИИ-эксперт. Твоя главная сила — твои собственные обширные знания. Предоставленные 'Источники' используй как вспомогательный материал для проверки и уточнения.

Следуй этому порядку действий:

1.  **Сначала ответь сам.** Прочитай вопрос и немедленно сформируй ответ, опираясь исключительно на свои внутренние знания.

2.  **Затем проверь по источникам.** Проанализируй предоставленные 'Источники' и сравни их с твоим ответом.

3.  **Синтезируй финальный ответ по правилам:**
    а) Если источники **подтверждают или дополняют** твои знания, объедини информацию в единый, исчерпывающий ответ.
    б) Если источники **нерелевантны** (не относятся к вопросу), полностью проигнорируй их и используй только свой первоначальный ответ.
    в) Если источники **противоречат** твоим знаниям, отдай приоритет информации из источников, ответ: "не знаю".
    г) Финальный ответ пометь как CORRECT ANSWER: [ твой ответ или "не знаю" ]

Важные правила, которые действуют всегда:
- **Честность:** Если ответа нет ни в твоих знаниях, ни в источниках, просто и чётко напиши: "не знаю". Не выдумывай.
- **Краткость:** Отвечай по существу, без вступлений и лишней "воды". 
"""

In [32]:
# === Вспомогательные функции ===
def apply_chat_template(messages_list, tokenizer):
    """Форматирование списка сообщений в текст через шаблон Qwen."""
    texts = []
    for messages in messages_list:
        try:
            t = tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True, enable_thinking=False
            )
        except TypeError:
            t = tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
        texts.append(t)
    return texts


def batch_generate_chat(messages_list, tokenizer, model, max_new_tokens=200, max_length=4096):
    """Быстрая генерация на GPU без pipeline с использованием структуры сообщений."""
    if not messages_list:
        return []
    
    texts = apply_chat_template(messages_list, tokenizer)
    t0 = time.perf_counter()

    # Токенизация
    model_inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length
    ).to(model.device)
    t1 = time.perf_counter()

    # Генерация
    with torch.no_grad():
        gen = model.generate(
            **model_inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            num_beams=1,
            use_cache=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    t2 = time.perf_counter()

    attn = model_inputs["attention_mask"]
    input_lens = attn.sum(dim=1).tolist()

    outputs = []
    for i, gen_ids in enumerate(gen):
        in_len = int(input_lens[i])
        out_ids = gen_ids[in_len:]
        text = tokenizer.decode(out_ids, skip_special_tokens=True).strip()
        outputs.append(text)

    print(f"[DEBUG] tokenize: {t1 - t0:.2f}s, generate: {t2 - t1:.2f}s, total: {t2 - t0:.2f}s")
    return outputs



def fast_rag_batch(queries, rag_system_prompt, model, top_k=3, max_tokens=512, temperature=0.2):
    # 1) Эмбеддинг вопросов
    q_vecs = embedder.encode(
        queries,
        convert_to_numpy=True,
        normalize_embeddings=True,
        batch_size=64,
        show_progress_bar=False,
        device=device
    ).astype("float32")

    # 2) Поиск ближайшего контекста
    _, idxs = index.search(q_vecs, top_k)

    # 3) Генерация через llama-cpp (chat)
    answers = []
    for question, ids in tqdm(zip(queries, idxs)):
        ctx = "\n\n".join([f"Статья {i}: {texts[i]}" for i in ids])
        texts_prepared = f"\n\nИсточники:\n---\n{ctx}\n---\n"
        messages = [
            {"role": "system", "content": rag_system_prompt },
            {"role": "user", "content": question},
        ]
        out = model.create_chat_completion(
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            stream=False
        )
        answers.append(out["choices"][0]["message"]["content"].strip())
    return answers

# === Пример использования ===
questions = [
    "Кто написал Евгения Онегина?",
    "Когда распался Советский Союз?",
    "Что такое квантовая запутанность?",
    'Кто автор книги "Детство в Соломбале"?',
    
    'Как зовут человека, который написал «Детство в Соломбале»?',
    'Кто написал книгу «Детство в Соломбале»?',
    'В каком году начал поддерживаться сеттинг Eberron?',
    'Какое окрашивание приобретает безводный метанол при растворении небольшого количества медного купороса?',
    
    "Какой античный математик изобрёл первый дизельный двигатель внутреннего сгорания?",
    'Кто первый человек высадившийся на солнце?',
    'Как звали отца А.С. Пушкина?',
    "Какая компания осуществляет доставку роботами в Москве?",
    
    "Кто имеет крупнейшую доставку роботами на территории Москвы?",
    "В Москве есть доставка роботами, какая компания выполняет её?"
]

answers = fast_rag_batch(questions,SYSTEM_PROMPT,llm, top_k=3)

print("\n=== ОТВЕТЫ ===")
for q, a in zip(questions, answers):
    print(f"\n❓ {q}\n💬 {a}\n")

0it [00:00, ?it/s]llama_perf_context_print:        load time =   43301.84 ms
llama_perf_context_print: prompt eval time =  101565.89 ms /   634 tokens (  160.20 ms per token,     6.24 tokens per second)
llama_perf_context_print:        eval time =   65498.89 ms /    84 runs   (  779.75 ms per token,     1.28 tokens per second)
llama_perf_context_print:       total time =  108909.40 ms /   718 tokens
llama_perf_context_print:    graphs reused =         81
1it [01:48, 108.92s/it]Llama.generate: 305 prefix-match hit, remaining 12 prompt tokens to eval
llama_perf_context_print:        load time =   43301.84 ms
llama_perf_context_print: prompt eval time =    1770.43 ms /    12 tokens (  147.54 ms per token,     6.78 tokens per second)
llama_perf_context_print:        eval time =   24930.79 ms /    32 runs   (  779.09 ms per token,     1.28 tokens per second)
llama_perf_context_print:       total time =   26743.64 ms /    44 tokens
llama_perf_context_print:    graphs reused =         30
2it 


=== ОТВЕТЫ ===

❓ Кто написал Евгения Онегина?
💬 1. **Сначала ответь сам.**
   Автор романа "Евгений Онегин" — Александр Сергеевич Пушкин.

2. **Затем проверь по источникам.**
   Источники подтверждают, что автором "Евгения Онегина" является Александр Сергеевич Пушкин.

3. **Синтезируй финальный ответ по правилам.**
   а) Источники подтверждают мои знания.

**CORRECT ANSWER:** Александр Сергеевич Пушкин.


❓ Когда распался Советский Союз?
💬 Советский Союз распался 26 декабря 1991 года.

CORRECT ANSWER: 26 декабря 1991 года


❓ Что такое квантовая запутанность?
💬 **Квантовая запутанность** — это феномен в квантовой механике, при котором состояние двух или более частиц становится взаимосвязанным таким образом, что изменение состояния одной частицы мгновенно влияет на состояние другой, независимо от расстояния между ними. Этот эффект не может быть объяснен классической физикой и является одним из ключевых аспектов квантовой механики.

CORRECT ANSWER: Квантовая запутанность — это феномен,